In [1]:
import uuid 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("COnsumeVendas") \
    .master('spark://spark-master:7077') \
    .config("spark.extraListeners", "io.openlineage.spark.agent.OpenLineageSparkListener") \
    .config("spark.openlineage.columnLineage.datasetLineageEnabled", "true") \
    .config("spark.openlineage.transport.type", "http") \
    .config("spark.openlineage.transport.url", "http://api:5000") \
    .config("spark.openlineage.namespace", "spark_integration") \
    .config("spark.openlineage.parentJobName", "customer-job") \
    .config("spark.openlineage.parentRunId", uuid.uuid4().__str__()) \
    .getOrCreate()

25/06/19 18:13:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
delta_path_vendas = "s3a://gold/compras_cliente/"
# Ler os dados
df_vendas = spark.read.format("delta").load(delta_path_vendas)

25/06/19 18:13:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [3]:
df_vendas.show()

25/06/19 18:13:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 9:>                                                          (0 + 1) / 1]

+-------------+------------------+
|customer_name|       total_price|
+-------------+------------------+
|          Ana| 151.9000015258789|
|       Carlos| 170.4699993133545|
|         João|211.73999786376953|
|        Lucas|191.49999618530273|
|        Maria|247.02000045776367|
+-------------+------------------+



In [4]:
# Configurações de conexão PostgreSQL
postgres_db = "jdbc:postgresql://dw:5432/postgres"
usuario = "dw"
senha = "postgres"


# Gravar no PostgreSQL
df_vendas.write \
    .format("jdbc") \
    .option("url", postgres_db) \
    .option("dbtable", "public.compras_clientes") \
    .option("user", usuario) \
    .option("password", senha) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

25/06/19 18:14:00 ERROR ContextFactory: Query execution is null: can't emit event for executionId 7
25/06/19 18:14:00 ERROR ContextFactory: Query execution is null: can't emit event for executionId 7
25/06/19 18:14:00 ERROR ContextFactory: Query execution is null: can't emit event for executionId 9
25/06/19 18:14:00 ERROR ContextFactory: Query execution is null: can't emit event for executionId 9
25/06/19 18:14:00 ERROR ContextFactory: Query execution is null: can't emit event for executionId 10
25/06/19 18:14:00 ERROR ContextFactory: Query execution is null: can't emit event for executionId 10
25/06/19 18:14:03 ERROR ContextFactory: Query execution is null: can't emit event for executionId 11
25/06/19 18:14:03 ERROR ContextFactory: Query execution is null: can't emit event for executionId 11
25/06/19 18:14:03 ERROR ContextFactory: Query execution is null: can't emit event for executionId 12
25/06/19 18:14:03 ERROR ContextFactory: Query execution is null: can't emit event for execution

In [5]:
spark.stop()